<a href="https://colab.research.google.com/github/MStamirski/Spaceship-Titanic/blob/main/Model_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive/MyDrive/Colab_Notebooks/SDA_upskill/Spaceship"

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/SDA_upskill/Spaceship


In [ ]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Model with parameters

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
model = XGBClassifier(eval_metric='logloss')

In [ ]:
def objective(trial):

    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0),
        'eval_metric': 'logloss'
    }

    optuna_model = XGBClassifier(**params)
    optuna_model.fit(X_train, y_train)

    y_pred = optuna_model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)

    return accuracy

# Features datasets

In [ ]:
!pip install import-ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.9 MB/s eta 0:00:00


In [ ]:
import import_ipynb

In [ ]:
from FeaturesEngineering import get_features, categories_one_hot_encoding, categories_target_encoding, categories_leave_one_out_encoding

importing Jupyter notebook from FeaturesEngineering.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 KB 3.4 MB/s eta 0:00:00


In [ ]:
df = get_features('train')
df_ohe = categories_one_hot_encoding(df)
df_te = categories_target_encoding(df)
df_looe = categories_leave_one_out_encoding(df)

# Initial verification

In [ ]:
from Optimization import verify_feature_dataset, get_subsets, model_optimization, test_tuned_model

importing Jupyter notebook from Optimization.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 11.2 MB/s eta 0:00:00


In [ ]:
acc_ohe_ini = verify_feature_dataset(df_ohe, model)

Accuracy: 78.02914110429448%


In [ ]:
acc_te_ini = verify_feature_dataset(df_te, model)

Accuracy: 76.95552147239265%


In [ ]:
acc_looe_ini = verify_feature_dataset(df_looe, model)

Accuracy: 76.61042944785275%


# Parameters tuning

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = get_subsets(df_ohe)
params = model_optimization(20, objective)
model = XGBClassifier(**params)
result_ohe_tuned = test_tuned_model(model, X_train, y_train, X_test, y_test)

[I 2023-04-08 19:37:08,217] A new study created in memory with name: no-name-b3d9cf16-a422-4829-9479-7a51337b006e
[I 2023-04-08 19:37:12,996] Trial 0 finished with value: 0.7821710999281093 and parameters: {'max_depth': 2, 'learning_rate': 0.8905989968335102, 'n_estimators': 462, 'min_child_weight': 4, 'gamma': 0.3162337168034115, 'subsample': 0.7858586596439272, 'colsample_bytree': 0.4012179959089981, 'reg_alpha': 0.9607783331972649, 'reg_lambda': 0.04585906047982418}. Best is trial 0 with value: 0.7821710999281093.
[I 2023-04-08 19:37:25,205] Trial 1 finished with value: 0.790079079798706 and parameters: {'max_depth': 5, 'learning_rate': 0.14929453888229932, 'n_estimators': 440, 'min_child_weight': 8, 'gamma': 0.007467049642029866, 'subsample': 0.9866137308282651, 'colsample_bytree': 0.8990280546355041, 'reg_alpha': 0.1630704647484143, 'reg_lambda': 0.3224200367213509}. Best is trial 1 with value: 0.790079079798706.
[I 2023-04-08 19:37:30,106] Trial 2 finished with value: 0.790079079


Number of finished trials: 20
Best trial:
Value: 0.8109273903666427
  Params: 
    max_depth: 2
    learning_rate: 0.28450184096326414
    n_estimators: 161
    min_child_weight: 7
    gamma: 0.5290190005676925
    subsample: 0.6522134718485615
    colsample_bytree: 0.46660536604485203
    reg_alpha: 0.6346725321572362
    reg_lambda: 0.3319264198360406

Accuracy after tuning: 78.953421506613%

Classification report
              precision    recall  f1-score   support

       False       0.80      0.77      0.78       861
        True       0.78      0.81      0.80       878

    accuracy                           0.79      1739
   macro avg       0.79      0.79      0.79      1739
weighted avg       0.79      0.79      0.79      1739


Confusion matrix
[[661 200]
 [166 712]]

All features number: 90, Selected features number: 23
Selected_features:

HomePlanet_Earth
HomePlanet_Europa
CryoSleep_False
CryoSleep_True
Destination_55 Cancri e
Cabin_deck_E
Cabin_deck_G
Cabin_side_S
Cabin_n

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = get_subsets(df_te)
params = model_optimization(20, objective)
model = XGBClassifier(**params)
result_te_tuned = test_tuned_model(model, X_train, y_train, X_test, y_test)

[I 2023-04-08 19:37:58,422] A new study created in memory with name: no-name-a868e30d-0d96-4a74-9b02-2e1bed9c16d3
[I 2023-04-08 19:37:58,650] Trial 0 finished with value: 0.7872034507548527 and parameters: {'max_depth': 1, 'learning_rate': 0.29714135153671983, 'n_estimators': 205, 'min_child_weight': 10, 'gamma': 0.9672194949767091, 'subsample': 0.49589333777477024, 'colsample_bytree': 0.4638997382743321, 'reg_alpha': 0.21262760910805198, 'reg_lambda': 0.5585324819475308}. Best is trial 0 with value: 0.7872034507548527.
[I 2023-04-08 19:37:59,868] Trial 1 finished with value: 0.7706685837526959 and parameters: {'max_depth': 5, 'learning_rate': 0.49112520704754525, 'n_estimators': 352, 'min_child_weight': 5, 'gamma': 0.08522749367424255, 'subsample': 0.6382301199562852, 'colsample_bytree': 0.7354035577585725, 'reg_alpha': 0.7711256491284718, 'reg_lambda': 0.22797610730392556}. Best is trial 0 with value: 0.7872034507548527.
[I 2023-04-08 19:38:00,237] Trial 2 finished with value: 0.7620


Number of finished trials: 20
Best trial:
Value: 0.8023005032350827
  Params: 
    max_depth: 8
    learning_rate: 0.2970586427741798
    n_estimators: 296
    min_child_weight: 2
    gamma: 0.2576939221821858
    subsample: 0.9662874973813795
    colsample_bytree: 0.28279804074495146
    reg_alpha: 0.839829659044996
    reg_lambda: 0.030383086941659615

Accuracy after tuning: 78.205865439908%

Classification report
              precision    recall  f1-score   support

       False       0.79      0.77      0.78       861
        True       0.78      0.79      0.79       878

    accuracy                           0.78      1739
   macro avg       0.78      0.78      0.78      1739
weighted avg       0.78      0.78      0.78      1739


Confusion matrix
[[663 198]
 [181 697]]

All features number: 14, Selected features number: 3
Selected_features:

CryoSleep_transformed
Spa_deciles_transformed
VRD_deciles_transformed


In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = get_subsets(df_looe)
params = model_optimization(20, objective)
model = XGBClassifier(**params)
result_looe_tuned = test_tuned_model(model, X_train, y_train, X_test, y_test)

[I 2023-04-08 19:38:14,599] A new study created in memory with name: no-name-9b9bdd6e-8583-439b-a7c0-04b018191c7b
[I 2023-04-08 19:38:15,089] Trial 0 finished with value: 0.7850467289719626 and parameters: {'max_depth': 3, 'learning_rate': 0.022467684809623318, 'n_estimators': 357, 'min_child_weight': 8, 'gamma': 0.24820713739772213, 'subsample': 0.12260223542264276, 'colsample_bytree': 0.24876926497909196, 'reg_alpha': 0.1545871134637757, 'reg_lambda': 0.5925837065290229}. Best is trial 0 with value: 0.7850467289719626.
[I 2023-04-08 19:38:15,480] Trial 1 finished with value: 0.6693026599568656 and parameters: {'max_depth': 8, 'learning_rate': 0.9980662026044518, 'n_estimators': 448, 'min_child_weight': 4, 'gamma': 0.7158447564982988, 'subsample': 0.03887408044917695, 'colsample_bytree': 0.040696465119759925, 'reg_alpha': 0.38530769529839737, 'reg_lambda': 0.824800719847644}. Best is trial 0 with value: 0.7850467289719626.
[I 2023-04-08 19:38:18,242] Trial 2 finished with value: 0.769


Number of finished trials: 20
Best trial:
Value: 0.7850467289719626
  Params: 
    max_depth: 3
    learning_rate: 0.022467684809623318
    n_estimators: 357
    min_child_weight: 8
    gamma: 0.24820713739772213
    subsample: 0.12260223542264276
    colsample_bytree: 0.24876926497909196
    reg_alpha: 0.1545871134637757
    reg_lambda: 0.5925837065290229

Accuracy after tuning: 77.17078780908568%

Classification report
              precision    recall  f1-score   support

       False       0.77      0.77      0.77       861
        True       0.77      0.77      0.77       878

    accuracy                           0.77      1739
   macro avg       0.77      0.77      0.77      1739
weighted avg       0.77      0.77      0.77      1739


Confusion matrix
[[662 199]
 [198 680]]

All features number: 14, Selected features number: 5
Selected_features:

CryoSleep_transformed
RService_deciles_transformed
ShMall_deciles_transformed
Spa_deciles_transformed
VRD_deciles_transformed


# Save results

In [ ]:
data = []
data.append(['XGBoostClassifier', 'OHE', acc_ohe_ini, result_ohe_tuned[0], str(result_ohe_tuned[1])+" / 90"])
data.append(['XGBoostClassifier', 'TE', acc_te_ini, result_te_tuned[0], str(result_te_tuned[1])+" / 14"])
data.append(['XGBoostClassifier', 'LOOE', acc_looe_ini, result_looe_tuned[0], str(result_looe_tuned[1])+" / 14"])

In [ ]:
import csv
from os.path import exists
resfile = 'spaceship_results.csv'

In [ ]:
if exists(resfile):
  f = open(resfile, 'a')
  writer = csv.writer(f)
else:
  header = ['Model', 'Categories_encoding', 'Initial_accuracy', 'Tuned_Accuracy', 'Important_Features']
  f = open(resfile, 'w', newline='')
  writer = csv.writer(f)
  writer.writerow(header)

writer.writerows(data)
f.close()